In [95]:
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
census = pd.read_csv('/home/umairshah/Datasets/census_data.csv')

In [3]:
census.head(3) # Reports back the Data frame of citizens with features like age, occupation
# and hours_per_week

# Our task is, on the basis of given features of the citizens, we need to 
# figure out whether their incomes is more than $50k or less then $50k

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [4]:
census['income_bracket'].nunique()

2

In [5]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [6]:
def convert(string):
    if string == ' <=50K':
        return 0
    else:
        return 1

In [7]:
census['income_bracket'] = census['income_bracket'].apply(convert)

In [8]:
census['income_bracket'].unique()

array([0, 1])

In [23]:
census.head(3)

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0


In [24]:
X = census.drop('income_bracket', axis = 1)
y = census['income_bracket']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

In [31]:
X_train.head(3)

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
20895,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Female,0,0,28,United-States
3384,47,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Wife,Black,Female,15024,0,40,United-States
1832,46,Local-gov,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,24,United-States


In [49]:
# Steps: 1) Creating featur columns
# Transform the existing columns into either categorical_column_with_vocabulary_list
# or hash_buckets
# Remember: Hash_bucketing and categorical_column_with_vocabulary_list are only for
# categorical columns
# for continuous columns we need to create feature columns using feature_column.numerica_col
# class from tensorflow

gender = tf.feature_column.categorical_column_with_vocabulary_list('gender', ['Female', 'Male'])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size = 1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size = 1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size = 1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size = 1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size = 1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size = 1000)

In [52]:
# For continuous columns

age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [58]:
feat_cols = [gender, occupation, marital_status, relationship, education, workclass,
            native_country, age, education_num, capital_gain, hours_per_week]

In [59]:
feat_cols

[VocabularyListCategoricalColumn(key='gender', vocabulary_list=('Female', 'Male'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 HashedCategoricalColumn(key='occupation', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='marital_status', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='relationship', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='education', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='workclass', hash_bucket_size=1000, dtype=tf.string),
 HashedCategoricalColumn(key='native_country', hash_bucket_size=1000, dtype=tf.string),
 NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='education_num', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='capital_gain', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='hours_per_week', sha

In [71]:
# Step: 02) Creating an Training input function

train_input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size = 32,
                                                num_epochs = 10, shuffle = True)

In [66]:
# Step: 03) Creating the model
# Remember! If we want to use DNNClassifier as our model, we need to convert all the 
# categorical columns we just created into embedded_categorical_columns, passing all
# of the categorical columns into the embedding function

model = tf.estimator.LinearClassifier(feature_columns = feat_cols)

W0906 16:17:00.818210 140306527729472 estimator.py:1811] Using temporary folder as model directory: /tmp/tmp0ota6n3x


In [67]:
# Step: 04) Training the model

model.train(input_fn = input_func, steps = 1000)

In [72]:
# Step: 05) Creating Test input function
test_input_func = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size = 32,
                                             num_epochs = 1, shuffle = False)

In [82]:
# Step: 06) Creating the Predictions from model

pred_gen = model.predict(input_fn = test_input_func)

In [83]:
predictions = list(pred_gen)

In [89]:
final_predictions = [pred['class_ids'][0] for pred in predictions]

In [94]:
# So here's the result of our LinearClassifier model
# 1's represent people having more then $50K income
# while 0's represent people having less than or equals to $50K income
# I'm slicing only first 20 values, you can run the code and check our the whole list

final_predictions[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]

In [96]:
print(confusion_matrix(y_test, final_predictions))
print('\n')
print(classification_report(y_test, final_predictions))

[[7076  360]
 [1507  826]]


              precision    recall  f1-score   support

           0       0.82      0.95      0.88      7436
           1       0.70      0.35      0.47      2333

   micro avg       0.81      0.81      0.81      9769
   macro avg       0.76      0.65      0.68      9769
weighted avg       0.79      0.81      0.78      9769



In [97]:
# Around 80% accuracy which is not bad!!
# You can go ahead and change the parameters, training data, num_epochs and may be number
# of steps to see the variations in accuracy!
# Thank you